In [8]:
import glob, os
from functools import reduce
import pandas as pd
import pycountry

RAW_WB    = "data/raw/worldbank"
RAW_EXT   = "data/raw/external"
CLEAN_DIR = "data/clean"
os.makedirs(CLEAN_DIR, exist_ok=True)

In [9]:
wb_paths  = glob.glob(f"{RAW_WB}/*.csv")
ext_paths = glob.glob(f"{RAW_EXT}/*.csv")
all_paths = wb_paths + ext_paths

print("Worldbank files:", wb_paths)
print("External files: ", ext_paths)

if not all_paths:
    raise FileNotFoundError("No CSVs found in worldbank/ or external/.")

Worldbank files: ['data/raw/worldbank\\gdp_per_capita_ppp.csv', 'data/raw/worldbank\\gdp_total_usd.csv', 'data/raw/worldbank\\life_expectancy.csv', 'data/raw/worldbank\\military_expenditure_pct_gdp.csv', 'data/raw/worldbank\\population_density.csv', 'data/raw/worldbank\\population_growth_pct.csv', 'data/raw/worldbank\\real_gdp_growth_pct.csv', 'data/raw/worldbank\\total_population.csv', 'data/raw/worldbank\\unemployment_rate.csv', 'data/raw/worldbank\\urbanization_rate.csv']
External files:  ['data/raw/external\\Corruption_Perception_Index_2024_iso3.csv', 'data/raw/external\\inflation_pct_wb.csv', 'data/raw/external\\literacy_rate_unesco.csv', 'data/raw/external\\military_expenditure_constusd.csv', 'data/raw/external\\wgi_gov_effectiveness.csv']


In [10]:
valid_iso3 = {c.alpha_3 for c in pycountry.countries}

frames = {}
for path in glob.glob(f"{RAW_WB}/*.csv") + glob.glob(f"{RAW_EXT}/*.csv"):
    ind = os.path.splitext(os.path.basename(path))[0]
    df  = pd.read_csv(path, usecols=["iso3","year","value"])
    
    # drop regions / aggregates
    df["iso3"] = df["iso3"].str.strip().str.upper()
    df = df[df["iso3"].isin(valid_iso3)]
    
    # keep each country’s most-recent row
    df2 = (
        df.sort_values("year")
          .drop_duplicates(subset="iso3", keep="last")
          .drop(columns="year")
          .rename(columns={"value": ind})
    )
    
    frames[ind] = df2
    print(f"{ind:<30} rows (countries only): {len(df2)}")

gdp_per_capita_ppp             rows (countries only): 215
gdp_total_usd                  rows (countries only): 215
life_expectancy                rows (countries only): 215
military_expenditure_pct_gdp   rows (countries only): 215
population_density             rows (countries only): 215
population_growth_pct          rows (countries only): 215
real_gdp_growth_pct            rows (countries only): 215
total_population               rows (countries only): 215
unemployment_rate              rows (countries only): 215
urbanization_rate              rows (countries only): 215
Corruption_Perception_Index_2024_iso3 rows (countries only): 166
inflation_pct_wb               rows (countries only): 175
literacy_rate_unesco           rows (countries only): 95
military_expenditure_constusd  rows (countries only): 154
wgi_gov_effectiveness          rows (countries only): 215


In [11]:
df_list = list(frames.values())

if len(df_list) == 1:
    master = df_list[0].copy()
else:
    master = df_list[0].copy()
    for df in df_list[1:]:
        master = master.merge(df, on="iso3", how="outer")

print("Merged shape:", master.shape)
master.head(50)

Merged shape: (216, 16)


,iso3,gdp_per_capita_ppp,gdp_total_usd,life_expectancy,military_expenditure_pct_gdp,population_density,population_growth_pct,real_gdp_growth_pct,total_population,unemployment_rate,urbanization_rate,Corruption_Perception_Index_2024_iso3,inflation_pct_wb,literacy_rate_unesco,military_expenditure_constusd,wgi_gov_effectiveness
0,ABW,44967.344513,3.648573e+09,76.353000,NaN,596.166667,0.045652,4.263719,1.073590e+05,NaN,44.254,NaN,NaN,NaN,NaN,0.795441
1,AFG,2211.280635,1.723305e+10,66.035000,NaN,62.215541,2.135594,2.710887,4.145476e+07,13.295,26.933,17.0,-6.601186,37.000000,2.304000e+10,-1.987014
2,AGO,8040.702450,8.482465e+10,64.617000,1.332529,28.583484,3.080655,1.001289,3.674991e+07,14.464,68.688,32.0,28.240495,72.400002,8.548000e+11,-1.008910
3,ALB,21263.195659,2.354718e+10,79.602000,1.743992,101.375511,-1.148418,3.936625,2.745972e+06,10.250,64.603,42.0,2.214490,98.500000,4.025600e+10,0.250855
4,AND,71730.668682,3.785067e+09,84.041000,NaN,169.585106,1.433748,2.583555,8.085600e+04,NaN,87.774,NaN,NaN,NaN,NaN,1.475439
5,ARE,76110.384846,5.141304e+11,82.909000,NaN,141.861124,3.977171,3.618707,1.048375e+07,2.133,87.779,68.0,1.626708,98.000000,NaN,1.604282
6,ARG,30082.304525,6.460753e+11,77.395000,0.472747,16.592272,0.286976,-1.611002,4.553840e+07,7.876,92.463,37.0,NaN,NaN,8.266490e+11,-0.377517
7,ARM,21342.514533,2.408575e+10,77.465854,5.450925,104.292237,0.728179,8.300000,2.990900e+06,13.329,63.739,47.0,0.269512,NaN,5.205200e+11,-0.184130
8,ASM,NaN,NaN,72.852000,NaN,241.710000,-1.712903,NaN,4.752100e+04,NaN,87.235,NaN,NaN,NaN,NaN,0.654168
9,ATG,32149.140365,2.033085e+09,77.598000,NaN,211.000000,0.511400,3.862012,9.331600e+04,NaN,24.332,NaN,5.067139,NaN,NaN,0.380411


In [12]:
import glob
import pandas as pd

# point to wherever your CSVs live
paths = glob.glob("data/raw/worldbank/*.csv") + glob.glob("data/raw/external/*.csv")

for path in paths:
    df = pd.read_csv(path)
    row_count = len(df)
    na_pct    = df.isna().mean().mul(100).round(1)   # percent missing, rounded to 1 decimal
    
    filename = os.path.basename(path)
    print(f"\n{filename} — rows: {row_count}")
    print(na_pct.astype(str) + "%")


gdp_per_capita_ppp.csv — rows: 266
country    0.0%
iso3       1.9%
year       0.0%
value      9.8%
dtype: object

gdp_total_usd.csv — rows: 266
country    0.0%
iso3       1.9%
year       0.0%
value      9.0%
dtype: object

life_expectancy.csv — rows: 266
country    0.0%
iso3       1.9%
year       0.0%
value      0.4%
dtype: object

military_expenditure_pct_gdp.csv — rows: 266
country     0.0%
iso3        1.9%
year        0.0%
value      27.1%
dtype: object

population_density.csv — rows: 266
country    0.0%
iso3       1.9%
year       0.0%
value      3.0%
dtype: object

population_growth_pct.csv — rows: 266
country    0.0%
iso3       1.9%
year       0.0%
value      0.4%
dtype: object

real_gdp_growth_pct.csv — rows: 266
country    0.0%
iso3       1.9%
year       0.0%
value      8.6%
dtype: object

total_population.csv — rows: 266
country    0.0%
iso3       1.9%
year       0.0%
value      0.4%
dtype: object

unemployment_rate.csv — rows: 266
country     0.0%
iso3        1.9%
year       

In [13]:
for col in master.columns.difference(["iso3"]):
    master[col] = master[col].fillna(master[col].median())

In [15]:
out_path = f"{CLEAN_DIR}/country_snapshot_master.csv"
master.to_csv(out_path, index=False)
print("✅ Saved:", out_path, "| rows:", len(master))

✅ Saved: data/clean/country_snapshot_master.csv | rows: 216
